## Imports

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

## Set Device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [6]:
in_channels = 3
num_classes = 10
learning_rate = 0.001
batch_size = 1024
num_epochs = 10
load_model = True

## Load Pretrained model and Modify it

In [15]:
## Pytorch has some inbuilt models which we can use
## If you are using same dataset for training then same pretrained weights can be used
## but here we are going to use CIFAR10 dataset, so need to modify the model
## to match dimensions
## We need to modify the avgpool to get output (1,1) and the classifier to get output classes = 10
import os

# One way to change classifier and avgpool
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
model = torchvision.models.vgg16(pretrained=True)
model.avgpool = Identity()     #as want to change complete avgpool, otherwise avgpool[layer] ie, avgpool[1]
#model.classifier[0] = nn.Linear(512, 10)   #512 : num_of_inputs of vgg, 10:out_classes of CIFAR
##Try
# model.classifier = nn.Sequential(nn.Linear(512,100),
#                                 nn.Dropout(p=0.5)
#                                 nn.Linear(100,10))
model.classifier = nn.Linear(512,10)
model.to(device)

##OR
# for i in range(1,7):
#     model.classifier[i] = Identity()
print(model)


# sys.exit()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Load Data

In [16]:
train_dataset = datasets.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

print('==Dataset Loaded==')

Files already downloaded and verified
==Dataset Loaded==


## Create FCN

In [17]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=10, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(1,1), stride=(1,1), padding=(1,1))
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc1 = nn.Linear(20*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
#         print('x shape after pool2: ',x.shape)

        x = x.reshape(x.shape[0], -1)
#         print('x shape after reshape: ',x.shape)
#         print(x.shape)
        return x

In [ ]:
## check
model = CNN(3,10)
x = torch.randn(74,3,32,32)
print(model(x).shape)

## Initialize the Network

In [18]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

## Loss and Optimizer

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Save and Load Checkpoints

In [25]:
def save_checkpoints(state, filename='save_checkpoint.pth.tar'):
    print("==> Saving checkpoints")
    torch.save(state, filename)
    
def load_checkpoints(checkpoint):
    print("==> Loading checkpoints")
    model.load_state_dict(checkpoint['model_check_dict'])          #dictionary keyname to save model checkpoint in training network phase
    optimizer.load_state_dict(checkpoint['optim_check_dict'])

## Load Checkpoint file


In [26]:
if load_model:
    load_checkpoints(torch.load('save_checkpoint.pth.tar'))

FileNotFoundError: [Errno 2] No such file or directory: 'save_checkpoint.pth.tar'

## Train Network

In [27]:
for epoch in range(num_epochs):
    losses = []
    
    if epoch % 2 == 0:
        checkpoint = {'model_check_dict': model.state_dict(), 'optim_check_dict':optimizer.state_dict()}
        save_checkpoints(checkpoint)
        
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
    pred = model(data)
    loss = criterion(pred, targets)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
mean_loss = sum(losses)/len(losses)
print(f'Loss at epoch {epoch} was {mean_loss:.5f}')

==> Saving checkpoints
==> Saving checkpoints
==> Saving checkpoints
==> Saving checkpoints
==> Saving checkpoints
Loss at epoch 9 was 6.99420


## Check Accuracy

In [28]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuaracy on Training data")
    else:
        print("Checking accuracy on Test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            pred = model(x)
            _, predictions = pred.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
            print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()

In [29]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuaracy on Training data
Got 44/1024 with accuracy 4.30
Got 76/2048 with accuracy 3.71
Got 117/3072 with accuracy 3.81
Got 150/4096 with accuracy 3.66
Got 187/5120 with accuracy 3.65
Got 230/6144 with accuracy 3.74
Got 272/7168 with accuracy 3.79
Got 303/8192 with accuracy 3.70
Got 338/9216 with accuracy 3.67
Got 369/10240 with accuracy 3.60
Got 419/11264 with accuracy 3.72
Got 459/12288 with accuracy 3.74
Got 491/13312 with accuracy 3.69
Got 530/14336 with accuracy 3.70
Got 572/15360 with accuracy 3.72
Got 603/16384 with accuracy 3.68
Got 637/17408 with accuracy 3.66
Got 674/18432 with accuracy 3.66
Got 717/19456 with accuracy 3.69
Got 760/20480 with accuracy 3.71
Got 808/21504 with accuracy 3.76
Got 844/22528 with accuracy 3.75
Got 885/23552 with accuracy 3.76
Got 925/24576 with accuracy 3.76
Got 962/25600 with accuracy 3.76
Got 998/26624 with accuracy 3.75
Got 1046/27648 with accuracy 3.78
Got 1099/28672 with accuracy 3.83
Got 1138/29696 with accuracy 3.83
Got 1181/30720 

NameError: name 'test_loader' is not defined